In [1]:
import torch
import torchvision.datasets as dsets
from torchvision import transforms
import random


# check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# set seed for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

torch.__version__, device

('1.7.0+cu101', 'cuda')

## Train CNN model with MNIST
- DNN과 마찬가지로 torch.nn.Module 상속해 모델 정의
- 각 CNN layer는 Conv2d + Relu + MaxPool2d로 구성
- 마지막 단계에서 view를 통한 flatten 작업 및 fc layer 통과

In [2]:
# set parameters
learning_rate = 0.001
n_epochs = 15
batch_size = 100

# prepare data
mnist_train = dsets.MNIST(root='mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='mnist_data/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader  = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw


Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2),
    )
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2),
    )
    self.fc = torch.nn.Linear(64*7*7, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    output = self.layer1(x)
    output = self.layer2(output)

    output = output.view(output.size(0), -1)
    output = self.fc(output)
    return output

In [4]:
%%time
# generate model
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# train model
total_batch = len(train_loader)
print('>>> Learning started. It takes sometime.')
for epoch in range(n_epochs):
  avg_cost = 0

  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    hypothesis = model(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print(f'[Epoch: {epoch+1:3}] cost = {avg_cost:10.6f}')
print('>>> Learning Finished!')


>>> Learning started. It takes sometime.
[Epoch:   1] cost =   0.220276
[Epoch:   2] cost =   0.060853
[Epoch:   3] cost =   0.045879
[Epoch:   4] cost =   0.036662
[Epoch:   5] cost =   0.030034
[Epoch:   6] cost =   0.026190
[Epoch:   7] cost =   0.020795
[Epoch:   8] cost =   0.018742
[Epoch:   9] cost =   0.015508
[Epoch:  10] cost =   0.013422
[Epoch:  11] cost =   0.011414
[Epoch:  12] cost =   0.008461
[Epoch:  13] cost =   0.008030
[Epoch:  14] cost =   0.006665
[Epoch:  15] cost =   0.007349
>>> Learning Finished!
CPU times: user 1min 15s, sys: 1.65 s, total: 1min 17s
Wall time: 1min 25s


In [5]:
# test model
with torch.no_grad():
  X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
  y_test = mnist_test.targets.float().to(device)
  pred = model(X_test)
  correct_pred = pred.argmax(1) == y_test
  acc = correct_pred.float().mean()
  print(f'>>> Test Accuracy: {acc:10.6f}')

>>> Test Accuracy:   0.986400
